In [24]:
from parcels import FieldSet, ParticleSet, Variable, JITParticle, AdvectionRK4, plotTrajectoriesFile, ErrorCode
import numpy as np
import scipy as sc
import math
from datetime import timedelta, datetime
from operator import attrgetter
import copy_plottrajectoriesfile as cpt
import matplotlib.animation as animation
import imageio #to make .gif out of multiple .png
import matplotlib.pyplot as plt
import scipy.io as sio #to load matlab file

"""
Load IMOS raw data
"""
filenames = "../raw_imos_current_data/IMOS_OceanCurrent_HV_2019_C-20190520T232835Z.nc"
variables = {'U' : 'UCUR',
                'V' : 'VCUR'}
dimensions = {'lat' : 'LATITUDE',
                  'lon' : 'LONGITUDE',
                  'time' : 'TIME'}
fieldset_imos = FieldSet.from_netcdf(filenames, variables, dimensions, deferred_load=False)

"""
Take TIME variable from the IMOS raw data
"""
from netCDF4 import Dataset
dataset=Dataset(filenames)
print(dataset.variables['TIME'])
time_imos=np.array(dataset.variables['TIME'])

<class 'netCDF4._netCDF4.Variable'>
float64 TIME(TIME)
    long_name: analysis time
    standard_name: time
    units: days since 1985-01-01 00:00:00+00:00
    coordinate_type: TIME
    calendar: gregorian
unlimited dimensions: TIME
current shape = (111,)
filling on, default _FillValue of 9.969209968386869e+36 used



In [25]:
fieldset_imos.U.data.shape

(111, 351, 641)

In [26]:
"""
Change U and V shape 
"""

lati=fieldset_imos.U.lat
longi=fieldset_imos.U.lon
ucur=fieldset_imos.U.data
vcur=fieldset_imos.V.data
u_imos=[]
v_imos=[]
grid_imos=[[],[]]

for i in range(len(lati)): #Create grid of IMOS latitude longitude
    for j in range(len(longi)):
        grid_imos[0].append(lati[i])
        grid_imos[1].append(longi[j]) 
grid_imos=np.asarray(grid_imos).T

for t in range(len(ucur)): #for every time step
    values_u=[]
    values_v=[]
    for i in range(len(lati)): #take U and V for every couple (lati,longi)
        for j in range(len(longi)):
            values_u.append(ucur[t,i,j])
            values_v.append(vcur[t,i,j])
    u_imos.append(np.asarray(values_u))
    v_imos.append(np.asarray(values_v))

"""
Create grid for extrapolated data (resolution 0.1 degree)
"""

grid_lat,grid_lon = np.mgrid[-24:-36:120j, 150:160:100j] 
grid_exp=(grid_lat,grid_lon)

In [27]:
"""
Extrapolate U and V data from grid_imos to grid_exp for each time step
"""

u_ext=[[] for i in range(len(u_imos))]
v_ext=[[] for i in range(len(v_imos))]
for i in range(len(u_imos)):
    u_ext[i] = sc.interpolate.griddata(grid_imos, u_imos[i], grid_exp, method='linear')
    v_ext[i] = sc.interpolate.griddata(grid_imos, v_imos[i], grid_exp, method='linear')
    print("number "+str(i)+" done")

number 0 done
number 1 done
number 2 done
number 3 done
number 4 done
number 5 done
number 6 done
number 7 done
number 8 done
number 9 done
number 10 done
number 11 done
number 12 done
number 13 done
number 14 done
number 15 done
number 16 done
number 17 done
number 18 done
number 19 done
number 20 done
number 21 done
number 22 done
number 23 done
number 24 done
number 25 done
number 26 done
number 27 done
number 28 done
number 29 done
number 30 done
number 31 done
number 32 done
number 33 done
number 34 done
number 35 done
number 36 done
number 37 done
number 38 done
number 39 done
number 40 done
number 41 done
number 42 done
number 43 done
number 44 done
number 45 done
number 46 done
number 47 done
number 48 done
number 49 done
number 50 done
number 51 done
number 52 done
number 53 done
number 54 done
number 55 done
number 56 done
number 57 done
number 58 done
number 59 done
number 60 done
number 61 done
number 62 done
number 63 done
number 64 done
number 65 done
number 66 done
numbe

In [8]:
"""
time_imos float to datetime object
"""

time=[]
for i in range(len(time_imos)):
    time.append(datetime.timestamp(datetime(1985,1,1,0,0,0)+timedelta(days=time_imos[i])))

In [9]:
datetime.fromtimestamp(time[0])

datetime.datetime(2019, 1, 1, 0, 0)

In [10]:
"""
Create fieldset with extrapolated data
"""

u_ext = np.array(u_ext,dtype=np.float64)
v_ext = np.array(v_ext,dtype=np.float64)
time_ext = np.array(time,dtype=np.float64)
data = {'U' : u_ext,
        'V' : v_ext}
dims = {'lon' : grid_lon,
        'lat' : grid_lat,
        'time' : time_ext}

fieldset_ext = FieldSet.from_data(data, dims, allow_time_extrapolation=True)

In [11]:
t

110

In [12]:
%matplotlib qt

In [23]:
"""
Compare before and after data extrapolation
"""
fieldset_imos.U.show(domain={'N':-24, 'S':-36, 'E':160, 'W':150},land=False,show_time=time_imos[0])
fieldset_ext.U.show(land=False,show_time=time_ext[0])

In [28]:
def DeleteParticle(particle, fieldset, time):
    particle.delete()

pset = ParticleSet.from_list(fieldset=fieldset_ext,
                             pclass=JITParticle,
                             lon=[151.265574],
                             lat=[-33.949705],
                             time=time[0])
#pset.show()
print(pset)

pset.execute(AdvectionRK4, #Only driven by currents
             runtime=timedelta(days=1), 
             dt=timedelta(minutes=5), #dt<0 : backwards tracking
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle}, #Delete particles when out of bounds
             output_file=pset.ParticleFile(name="TRY.nc", outputdt=timedelta(hours=1)))
#pset.show()
#print(pset)
#plotTrajectoriesFile('TRY.nc', mode='movie2d_notebook') #plot trajectory without the field on background

#images=[]
#for i in range(10):
#    pset.show(savefile='../outputs_imos_current_data/make_gif/particle_try_'+str(i),field='vector', vmin=0., vmax=1.5,land=False,show_time=time[i])
 #   images.append(imageio.imread('../outputs_imos_current_data/make_gif/particle_try_'+str(i)+'.png'))
#imageio.mimsave('../outputs_imos_current_data/particle_advection_reverse2.gif', images)

P[5](lon=151.265579, lat=-33.949703, depth=0.000000, time=1546261200.000000)


INFO: Compiled JITParticleAdvectionRK4 ==> /tmp/parcels-13529592/ea82347c0fa1f3e67d1212c1e1715f79.so


In [144]:
images=[] #list that will take all the .png outputs
pset.execute(AdvectionRK4, 
             runtime=timedelta(days=10), 
             dt=-timedelta(minutes=5), 
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

for i in range(10):
    pset.show(savefile='../outputs_imos_current_data/make_gif/particle_try_'+str(i),field='vector', vmin=0., vmax=1.5,land=False,show_time=time[i])
    images.append(imageio.imread('../outputs_imos_current_data/make_gif/particle_try_'+str(i)+'.png'))
imageio.mimsave('../outputs_imos_current_data/particle_advection_reverse2.gif', images)

INFO: Compiled JITParticleAdvectionRK4 ==> /tmp/parcels-13529592/07ffc8c1a4d8575ec39ae7b251574fea.so
INFO: Plot saved to ../outputs_imos_current_data/make_gif/particle_try_0.png
/home/z3529592/anaconda3/envs/py3_parcels/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1752: RuntimeWarning: invalid value encountered in less
  u, v = self.projection.transform_vectors(t, x, y, u, v)
/home/z3529592/anaconda3/envs/py3_parcels/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1752: RuntimeWarning: invalid value encountered in greater
  u, v = self.projection.transform_vectors(t, x, y, u, v)
INFO: Plot saved to ../outputs_imos_current_data/make_gif/particle_try_1.png
INFO: Plot saved to ../outputs_imos_current_data/make_gif/particle_try_2.png
INFO: Plot saved to ../outputs_imos_current_data/make_gif/particle_try_3.png
INFO: Plot saved to ../outputs_imos_current_data/make_gif/particle_try_4.png
INFO: Plot saved to ../outputs_imos_current_data/make_gif/particle_try_5.png
INFO: Plot saved to 

In [12]:
time[9]-time[8]

86400.0

In [14]:
def DeleteParticle(particle, fieldset, time):
    particle.delete()
import imageio
pset = ParticleSet.from_list(fieldset=fieldset_ext,
                             pclass=JITParticle,
                             lon=[151.2,  151.3],
                             lat=[-33.92, -33.9],
                             time=time[5])
images=[]
for i in range(48):
    pset.execute(AdvectionRK4, runtime=timedelta(hours=i), dt=-timedelta(minutes=5), recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})
    pset.show(savefile='../outputs_imos_current_data/make_gif/particle_try_'+str(i),field='vector', vmin=0., vmax=1.5,land=False,show_time=(time[5]+i*86400/24))
    images.append(imageio.imread('../outputs_imos_current_data/make_gif/particle_try_'+str(i)+'.png'))
imageio.mimsave('../outputs_imos_current_data/particle_advection_2days.gif', images)

INFO: Compiled JITParticleAdvectionRK4 ==> /tmp/parcels-13529592/6b9e330dd7caa9b2a594ed313fe036bd.so
/home/z3529592/anaconda3/envs/py3_parcels/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1752: RuntimeWarning: invalid value encountered in less
  u, v = self.projection.transform_vectors(t, x, y, u, v)
/home/z3529592/anaconda3/envs/py3_parcels/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1752: RuntimeWarning: invalid value encountered in greater
  u, v = self.projection.transform_vectors(t, x, y, u, v)
INFO: Plot saved to ../outputs_imos_current_data/make_gif/particle_try_0.png
INFO: Plot saved to ../outputs_imos_current_data/make_gif/particle_try_1.png
INFO: Plot saved to ../outputs_imos_current_data/make_gif/particle_try_2.png
INFO: Plot saved to ../outputs_imos_current_data/make_gif/particle_try_3.png
INFO: Plot saved to ../outputs_imos_current_data/make_gif/particle_try_4.png
INFO: Plot saved to ../outputs_imos_current_data/make_gif/particle_try_5.png
INFO: Plot saved to 

Add Brownian Motion Kernel

In [31]:
from parcels import (FieldSet, AdvectionRK4, BrownianMotion2D, Field,
                     ParticleSet, JITParticle, Variable, ErrorCode)
size2D = (fieldset_ext.U.grid.ydim, fieldset_ext.U.grid.xdim)
Kh_coeff=26.5 #meh 
fieldset_ext.add_field(Field('Kh_zonal', data=Kh_coeff*np.ones(size2D),
                             lon=fieldset_ext.U.grid.lon, lat=fieldset_ext.U.grid.lat,
                             mesh='spherical', allow_time_extrapolation=True))
fieldset_ext.add_field(Field('Kh_meridional', data=Kh_coeff*np.ones(size2D),
                             lon=fieldset_ext.U.grid.lon, lat=fieldset_ext.U.grid.lat,
                             mesh='spherical', allow_time_extrapolation=True))


In [32]:
from parcels import BrownianMotion2D

In [33]:
pset = ParticleSet.from_list(fieldset=fieldset_ext,
                             pclass=JITParticle,
                             lon=[155.6,  156.3],
                             lat=[-30, -30.9],
                             time=time[0])  
images=[]
kernels=pset.Kernel(BrownianMotion2D)+pset.Kernel(AdvectionRK4)
for i in range(20):
    pset.execute(kernels, runtime=timedelta(days=i), dt=timedelta(minutes=5), recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})
    pset.show(show_time=timedelta(days=i))
    print(pset)

INFO: Compiled JITParticleBrownianMotion2DAdvectionRK4 ==> /tmp/parcels-13529592/a5afc91921a879dd893445c7033d8beb.so


P[8](lon=155.600006, lat=-30.000000, depth=0.000000, time=1546261200.000000)
P[9](lon=156.300003, lat=-30.900000, depth=0.000000, time=1546261200.000000)
P[8](lon=155.441406, lat=-29.894695, depth=0.000000, time=1546347600.000000)
P[9](lon=156.128067, lat=-30.816671, depth=0.000000, time=1546347600.000000)
P[8](lon=155.358734, lat=-29.478491, depth=0.000000, time=1546520400.000000)
P[9](lon=155.763428, lat=-30.502600, depth=0.000000, time=1546520400.000000)
P[8](lon=155.867310, lat=-28.658531, depth=0.000000, time=1546779600.000000)
P[9](lon=154.860764, lat=-30.001019, depth=0.000000, time=1546779600.000000)
P[8](lon=157.034714, lat=-27.988018, depth=0.000000, time=1547125200.000000)
P[9](lon=155.424637, lat=-28.657713, depth=0.000000, time=1547125200.000000)
P[8](lon=158.294495, lat=-27.500162, depth=0.000000, time=1547557200.000000)
P[9](lon=157.009872, lat=-27.337095, depth=0.000000, time=1547557200.000000)
P[8](lon=159.066833, lat=-27.672092, depth=0.000000, time=1548075600.000000)

In [22]:
def NorthEast(particle, fieldset, time):
    particle.lon += 0.001*particle.dt
    particle.lat += 0.001*particle.dt

pset = ParticleSet.from_list(fieldset=fieldset_ext,
                             pclass=JITParticle,
                             lon=[151.6,  152.3],
                             lat=[-33, -33.9],
                             time=time[0])  
images=[]
kernels=pset.Kernel(AdvectionRK4)+pset.Kernel(NorthEast)+pset.Kernel(BrownianMotion2D)
for i in range(1):
    pset.execute(kernels, runtime=timedelta(hours=i), dt=timedelta(minutes=5), recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})
    pset.show(show_time=(time[0]+i*86400/24),savefile='../outputs_imos_current_data/make_gif/particle_kernel_try_'+str(i)+'.png', field='vector', vmin=0., vmax=1.5,land=False)
    images.append(imageio.imread('../outputs_imos_current_data/make_gif/particle_kernel_try_'+str(i)+'.png'))
imageio.mimsave('../outputs_imos_current_data/particle_advection_try.gif', images)

INFO: Compiled JITParticleAdvectionRK4NorthEastBrownianMotion2D ==> /tmp/parcels-13529592/ab33b84bcd90362f019c54e2cfa122ec.so
/home/z3529592/anaconda3/envs/py3_parcels/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1752: RuntimeWarning: invalid value encountered in less
  u, v = self.projection.transform_vectors(t, x, y, u, v)
/home/z3529592/anaconda3/envs/py3_parcels/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1752: RuntimeWarning: invalid value encountered in greater
  u, v = self.projection.transform_vectors(t, x, y, u, v)
INFO: Plot saved to ../outputs_imos_current_data/make_gif/particle_kernel_try_0.png.png
